In [ ]:
!pip install -q --upgrade google-generativeai langchain-google-genai python-dotenv

In [ ]:
import google.generativeai as genai
from google.colab import userdata

In [ ]:
GOOGLE_API_KEY = userdata.get('GeminiProKey')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
!echo -e 'GOOGLE_API_KEY=' > .env

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
!pip install langchain langchain-community langchain-experimental chromadb pandas

In [ ]:
!mkdir files

In [ ]:
!gdown "https://drive.google.com/file/d/1vqLfoWdCu-yv1fTiNxzBHB1wU-yNPalc/view?usp=sharing"

In [ ]:
import pandas as pd
df = pd.read_csv('top_500_albums.csv')
df.head()

In [41]:
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain_community.document_loaders import CSVLoader
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [25]:
loader = CSVLoader("top_500_albums.csv")
data = loader.load_and_split()

In [ ]:
print(data)

In [ ]:
print(data[0].page_content)

In [29]:
context = "\n".join(
    str(p.page_content) for p in data
)

In [ ]:
print("The total number of words in the context:", len(context))

In [31]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=15000, chunk_overlap=100)
context = "\n\n".join(
    str(p.page_content) for p in data
)

In [32]:
texts = text_splitter.split_text(context)

In [ ]:
print(len(texts))

In [36]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [37]:
vector_index = Chroma.from_texts(texts, embeddings).as_retriever()

In [ ]:
question = "Summarize the commentary from all the albums and suggest a new album based on the summary"
docs = vector_index.invoke(question)

In [ ]:
docs

In [65]:
prompt_template = """
    Responda a questão o mais detalhado possível a partir do contexto dado, tenha certeza de prover todos os detalhes.
    Se não há detalhes suficientes no contexto provido, você pode gerar respostas criativas.
    Não responda com informações erradas.\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
"""
prompt = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])

In [68]:
model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.8)

In [69]:
chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

In [70]:
response = chain(
    {
        "input_documents": docs,
        "question": question
    },
    return_only_outputs=True
)

In [71]:
response["output_text"]

'**Resumo dos Comentários:**\n\nOs álbuns apresentados são clássicos atemporais que demonstraram o auge criativo de seus artistas. Eles oferecem uma gama diversificada de gêneros, incluindo rock, rock progressivo, pop, hard rock e grunge. Esses álbuns influenciaram profundamente a música e a cultura popular, apresentando inovações em engenharia de som, narrativa envolvente e exploração de temas profundos.\n\n**Sugestão de Novo Álbum:**\n\nCom base nos resumos fornecidos, um novo álbum poderia abranger os seguintes aspectos:\n\n* **Gênero:** Rock Progressivo\n* **Artista:** Uma nova banda ou artista emergente com uma perspectiva única e inovadora.\n* **Título do Álbum:** "Ecos do Cosmos"\n* **Tema:** Explorar as vastas extensões do universo, contemplando questões de existência, mistério e o lugar da humanidade no cosmos.\n* **Estilo:** Uma fusão de rock progressivo com elementos de música eletrônica e influências étnicas, criando uma paisagem sonora imersiva e atmosférica.\n* **Narrativ